# **Finetuning VITS Text-to-Speech Model in Vietnamese**
@phatjk - Nguyễn Thành Phát
## (Nhấn CRTL + F9 để chạy tất cả)

# **Install packages**

In [1]:
# Setup Evironments
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy>=1.19.0 onnxruntime==1.15.1 pytorch-lightning==1.7.0 torch==1.11.0
!pip install -q torchtext==0.12.0 torchvision==0.12.0
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
!pip install -q textgrid
!bash build_monotonic_align.sh
!apt-get install -q espeak-ng

/content/piper/src/python
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 16.6 MB/s eta 0:00:00
Compiling /content/piper/src/python/piper_train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cyth

# **Download datasets**

In [2]:
!gdown 1Pe-5lKT_lZsliv2WxQDai2mjhI9ZMFlj
!unzip reinfo.zip -d wav/

Streaming output truncated to the last 5000 lines.
  inflating: wav/09498.wav           
  inflating: wav/08789.wav           
  inflating: wav/05317.TextGrid      
  inflating: wav/13339.TextGrid      
  inflating: wav/01585.wav           
  inflating: wav/08844.TextGrid      
  inflating: wav/05242.TextGrid      
  inflating: wav/03398.TextGrid      
  inflating: wav/02933.TextGrid      
  inflating: wav/08821.TextGrid      
  inflating: wav/13392.wav           
  inflating: wav/00707.TextGrid      
  inflating: wav/12514.wav           
  inflating: wav/06335.TextGrid      
  inflating: wav/04744.TextGrid      
  inflating: wav/05098.wav           
  inflating: wav/03930.wav           
  inflating: wav/03492.TextGrid      
  inflating: wav/14473.wav           
  inflating: wav/12696.wav           
  inflating: wav/00161.wav           
  inflating: wav/08032.TextGrid      
  inflating: wav/07103.wav           
  inflating: wav/06629.wav           
  inflating: wav/05475.TextGrid      

# **Data preprocessing**

In [3]:
import os
dir_path = r'wav'
res = []
# Lấy tất cả tên file
for file_path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, file_path)):
        res.append(file_path)

list_text_files = list(filter(lambda x: "TextGrid" in x,res))

# Chỉ cắt 1000 điểm dữ liệu để sử dụng finetuining
list_text_files = list_text_files[:1000]

# Lấy text của audio từ file TextGrid
import textgrid
all_id = []
all_text = []
for text_file in list_text_files:
  tg = textgrid.TextGrid.fromFile('wav/'+text_file)
  text = ""
  for i in range(len(tg[0])):
    if tg[0][i].mark != '':
      text += tg[0][i].mark + " "
  file_name = text_file.replace(".TextGrid","")
  all_id.append(file_name)
  all_text.append(text)

In [4]:
# Chuyển dữ liệu thành dạng dataframe để dễ xuất file
import pandas as pd

data = {
  "id": all_id,
  "text": all_text
}
df = pd.DataFrame(data)

print(df)

        id                                               text
0    13657  sổ hồng riêng mua bán công chứng ngay chủ nhà ...
1    03744  chuối và bơ hạnh nhân cũng được coi là thực ph...
2    13139  anh ấy nghĩ sai làm sai không gãy gánh sự nghi...
3    01294  hy vọng những dự định của thúy sẽ thành công đ...
4    08105  đây là phần nổi còn phần chìm dưới thì khó ai ...
..     ...                                                ...
995  08666  dù việc đi lại khó khăn viết và tiếp thu bài c...
996  13256  và khi đã chọn mua được tượng phật ưng ý chúng...
997  10059  nếu không điều tiết được tốc độ tăng đàn sẽ dẫ...
998  10914  trò chơi sẽ là một kho các câu hỏi và trả lời ...
999  05702  vốn là doanh nhân chuyên nghiệp các nhà đầu tư...

[1000 rows x 2 columns]


In [5]:
# Cắt tập chia tập train/test tỉ lệ 80/20
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

In [6]:
# Xuất file data train
train_df.to_csv('metadata.csv', sep='|', index=False,header=False)

In [7]:
# Xuất file text cho data text
import numpy as np
np.savetxt(r'test_vi.txt', test_df["text"].values, fmt='%s')

In [8]:
test_ds = test_df["text"].values

In [9]:
from piper_phonemize import (
    phonemize_espeak,
    phoneme_ids_espeak,
)
test_ds_phonemi = list(map(lambda x : phonemize_espeak(x, "vi"),test_ds))
test_ds_phonemi_ids = list(map(lambda x : phoneme_ids_espeak(x[0]),test_ds_phonemi))
test_ds_phonemi = list(map(lambda x:x[0],test_ds_phonemi))


In [10]:
data = {
  "text": test_ds,
  "phonemes": test_ds_phonemi,
  "phoneme_ids": test_ds_phonemi_ids
}
df_test_with_phonemi = pd.DataFrame(data)

df_test_with_phonemi

,text,phonemes,phoneme_ids
0,chiếc xe tải chở heo gây tai nạn bị chặn lại t...,"[t, ʃ, ˈ, i, ɛ, ɜ, c, , s, ˈ, ɛ, , t, ̪, ˈ, ...","[1, 0, 32, 0, 96, 0, 120, 0, 21, 0, 61, 0, 62,..."
1,cầu thủ người ai cập không ngừng theo đuổi sự ...,"[k, ˈ, ə, 2, w, , t, ˈ, u, 4, , ŋ, ˈ, y, ə, ...","[1, 0, 23, 0, 120, 0, 59, 0, 132, 0, 35, 0, 3,..."
2,shop mình hiện đang bán bột ngũ cốc bà bầu lợi...,"[ʃ, ˈ, ɒ, p, , m, ˈ, i, 2, ɲ, , h, ˈ, i, ɛ, ...","[1, 0, 96, 0, 120, 0, 52, 0, 28, 0, 3, 0, 25, ..."
3,cứ như vậy người bệnh tiếp tục ám ảnh sợ mất n...,"[k, ˈ, y, ɜ, , ɲ, ˌ, y, , v, ˈ, ə, ɪ, 6, , ...","[1, 0, 23, 0, 120, 0, 37, 0, 62, 0, 3, 0, 82, ..."
4,công nghiệp thì cần nhất là năng suất và tốc đ...,"[k, ˈ, o, ŋ, , ŋ, ˈ, i, ɛ, 6, p, , t, ˌ, i, ...","[1, 0, 23, 0, 120, 0, 27, 0, 44, 0, 3, 0, 44, ..."
...,...,...,...
195,luyện trau dồi đạo đức cách mạng tận tâm tận l...,"[l, w, ˈ, i, ɛ, 6, n, , t, ʃ, ˈ, a, w, , z, ...","[1, 0, 24, 0, 35, 0, 120, 0, 21, 0, 61, 0, 136..."
196,thông báo về việc đình chỉ dừng hội chợ khiến ...,"[t, ˈ, o, ŋ, , b, ˈ, a, ː, ɜ, w, , v, ˈ, e, ...","[1, 0, 32, 0, 120, 0, 27, 0, 44, 0, 3, 0, 15, ..."
197,lại trong một phỏng vấn khác lôi lôi nói đã kì...,"[l, ˈ, a, ː, 6, j, , t, ʃ, ˈ, ɔ, ŋ, , m, ˈ, ...","[1, 0, 24, 0, 120, 0, 14, 0, 122, 0, 136, 0, 2..."
198,thời gian gần đây tình trạng phân lô bán nền c...,"[t, ˈ, ə, ː, 2, j, , z, ˈ, a, ː, n, , ɣ, ˈ, ...","[1, 0, 32, 0, 120, 0, 59, 0, 122, 0, 132, 0, 2..."


In [11]:
df_test_with_phonemi.to_json('test_vi.jsonl', orient='records', lines=True)

In [12]:
!pwd

/content/piper/src/python


# **Resampling datasets**

In [13]:
# Đưa sampling rate của audio về 22050
resample = True
sample_rate = 22050
if resample:
  !python resample.py --input_dir "wav" --output_dir "wavs_resampled" --output_sr {sample_rate} --file_ext "wav"
  !mv wavs_resampled wav

Recursively copying the input folder...
Resampling the audio files...
Found 14935 files...
100% 14935/14935 [02:28<00:00, 100.90it/s]
Done !


# **Preprocess text to Phonemes**

In [14]:
# Chuyển dữ liệu text sang Phonemes (âm vị)
dataset_format = "ljspeech"
force_sp = "--single-speaker"
!python -m piper_train.preprocess \
  --language vi \
  --input-dir /content/piper/src/python \
  --output-dir /content/piper/src/python \
  --dataset-format {dataset_format} \
  --sample-rate {sample_rate} \
  {force_sp}

INFO:preprocess:Single speaker dataset
INFO:preprocess:Wrote dataset config
INFO:preprocess:Processing 800 utterance(s) with 2 worker(s)


# **Download pretrained model**

In [15]:
# Tải pretrain model để tiếng hành finetuning
!wget https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/vi/vi_VN/vais1000/medium/epoch%3D4769-step%3D919580.ckpt
!wget https://huggingface.co/datasets/rhasspy/piper-checkpoints/raw/main/vi/vi_VN/vais1000/medium/config.json

--2023-10-02 07:53:33--  https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/vi/vi_VN/vais1000/medium/epoch%3D4769-step%3D919580.ckpt
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.24, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/6c/3b/6c3bb73012be460f91a23801a60eb3831f76d24f6812c18e0ca05b16673ae99f/c3879268594a85b5133146e6a4709a5cc2f3014ef0eabf9280f67289054ba2bc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27epoch%253D4769-step%253D919580.ckpt%3B+filename%3D%22epoch%3D4769-step%3D919580.ckpt%22%3B&Expires=1696492413&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NjQ5MjQxM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Yy8zYi82YzNiYjczMDEyYmU0NjBmOTFhMjM4MDFhNjBlYjM4MzFmNzZkMjRmNjgxMmMxOGUwY2EwNWIxNjY3M2FlOTlmL

In [16]:
# Đổi tên pretrained model
!mv epoch=4769-step=919580.ckpt pretrained_vi.ckpt

# **Training model**

In [ ]:
#x-low - 16Khz audio, 5-7M params
#medium - 22.05Khz audio, 15-20 params
#high - 22.05Khz audio, 28-32M params
# checkpoint-epochs: Save checkpoint every N epochs (default: 1)

Cài đặt thông số và tiếng hành training

In [ ]:
!python -m piper_train \
--dataset-dir "/content/piper/src/python/" \
--accelerator 'gpu' \
--devices 1 \
--batch-size 34 \
--validation-split 0.0 \
--num-test-examples 2 \
--quality "medium" \
--checkpoint-epochs 5 \
--log_every_n_steps 50 \
--max_epochs 5000 \
--resume_from_checkpoint "/content/piper/src/python/pretrained_vi.ckpt"\
--precision 32

DEBUG:piper_train:Namespace(dataset_dir='/content/piper/src/python/', checkpoint_epochs=5, quality='medium', resume_from_single_speaker_checkpoint=None, logger=True, enable_checkpointing=True, default_root_dir=None, gradient_clip_val=None, gradient_clip_algorithm=None, num_nodes=1, num_processes=None, devices='1', gpus=None, auto_select_gpus=False, tpu_cores=None, ipus=None, enable_progress_bar=True, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=None, max_epochs=5000, min_epochs=None, max_steps=-1, min_steps=None, max_time=None, limit_train_batches=None, limit_val_batches=None, limit_test_batches=None, limit_predict_batches=None, val_check_interval=None, log_every_n_steps=50, accelerator='gpu', strategy=None, sync_batchnorm=False, precision=32, enable_model_summary=True, weights_save_path=None, num_sanity_val_steps=2, resume_from_checkpoint='/content/piper/src/python/pretrained_vi.ckpt', profiler=None, benchmark=None, de

# **View training information on Tensorboard**

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/piper/src/python/lightning_logs --port=8008

<IPython.core.display.Javascript object>

# **Export ONNX model**

In [ ]:
# Export onnx trên model finetuning (Nhớ đổi tên file cho đúng checkpoint)
!python3 -m piper_train.export_onnx \
    /content/piper/src/python/lightning_logs/version_0/checkpoints/epoch=4869-step=921980.ckpt \
    /content/finetuning_pretrained_vi.onnx

!cp /content/piper/src/python/config.json \
   /content/finetuning_pretrained_vi.onnx.json

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp6mgx9ntb
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp6mgx9ntb/_remote_module_non_sriptable.py
Removing weight norm...
/content/piper/src/python/piper_train/vits/attentions.py:235: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  t_s == t_t
/content/piper/src/python/piper_train/vits/attentions.py:295: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  pad_length = max(length - (self.window_size + 1), 0)
/content/piper/src/python/piper_train/vits/attentions.py:296:

In [32]:
# Export onnx trên model gốc
!python3 -m piper_train.export_onnx \
    /content/piper/src/python/pretrained_vi.ckpt \
    /content/pretrained_vi.onnx

!cp /content/piper/src/python/config.json \
   /content/pretrained_vi.onnx.json

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp7riv0joq
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp7riv0joq/_remote_module_non_sriptable.py
Removing weight norm...
/content/piper/src/python/piper_train/vits/attentions.py:235: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  t_s == t_t
/content/piper/src/python/piper_train/vits/attentions.py:295: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  pad_length = max(length - (self.window_size + 1), 0)
/content/piper/src/python/piper_train/vits/attentions.py:296:

# **Evaluation**

In [33]:
# Inference from .onnx file in original model on test data
!cat test_vi.jsonl | \
    python3 -m piper_train.infer_onnx \
        --sample-rate 22050 \
        --model /content/pretrained_vi.onnx \
        --output-dir test_wav_original

DEBUG:piper_train.infer_onnx:Loading model from /content/pretrained_vi.onnx
INFO:piper_train.infer_onnx:Loaded model from /content/pretrained_vi.onnx
DEBUG:piper_train.infer_onnx:Real-time factor for 1: 0.15 (infer=0.45 sec, audio=3.08 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 2: 0.14 (infer=0.53 sec, audio=3.80 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 3: 0.15 (infer=0.63 sec, audio=4.20 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 4: 0.14 (infer=0.76 sec, audio=5.28 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 5: 0.15 (infer=0.34 sec, audio=2.36 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 6: 0.15 (infer=0.57 sec, audio=3.89 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 7: 0.13 (infer=0.45 sec, audio=3.34 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 8: 0.14 (infer=0.63 sec, audio=4.37 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 9: 0.17 (infer=0.83 sec, audio=4.92 sec)
DEBUG:piper_train.infer_onnx:Re

In [35]:
!wget https://github.com/phatjkk/vits-tts-vietnamese/raw/main/fine-tuning-model/finetuning_pretrained_vi.onnx
!wget https://github.com/phatjkk/vits-tts-vietnamese/raw/main/fine-tuning-model/finetuning_pretrained_vi.onnx.json

--2023-10-02 08:31:41--  https://github.com/phatjkk/vits-tts-vietnamese/raw/main/fine-tuning-model/finetuning_pretrained_vi.onnx
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/phatjkk/vits-tts-vietnamese/main/fine-tuning-model/finetuning_pretrained_vi.onnx [following]
--2023-10-02 08:31:41--  https://raw.githubusercontent.com/phatjkk/vits-tts-vietnamese/main/fine-tuning-model/finetuning_pretrained_vi.onnx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63122309 (60M) [application/octet-stream]
Saving to: ‘finetuning_pretrained_vi.onnx’

finetuning_pretrain 100%[===================>]  60.20M   2

In [36]:
# Inference from .onnx file in finetuning model on test data
!cat test_vi.jsonl | \
    python3 -m piper_train.infer_onnx \
        --sample-rate 22050 \
        --model finetuning_pretrained_vi.onnx \
        --output-dir test_wav_finetuning

DEBUG:piper_train.infer_onnx:Loading model from finetuning_pretrained_vi.onnx
INFO:piper_train.infer_onnx:Loaded model from finetuning_pretrained_vi.onnx
DEBUG:piper_train.infer_onnx:Real-time factor for 1: 0.15 (infer=0.52 sec, audio=3.51 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 2: 0.14 (infer=0.66 sec, audio=4.76 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 3: 0.14 (infer=0.62 sec, audio=4.40 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 4: 0.14 (infer=0.85 sec, audio=6.01 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 5: 0.16 (infer=0.45 sec, audio=2.77 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 6: 0.20 (infer=1.00 sec, audio=4.96 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 7: 0.23 (infer=0.86 sec, audio=3.70 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 8: 0.23 (infer=1.09 sec, audio=4.67 sec)
DEBUG:piper_train.infer_onnx:Real-time factor for 9: 0.23 (infer=1.18 sec, audio=5.15 sec)
DEBUG:piper_train.infer_onn

In [39]:
import librosa
mse_original = 0
rmse_original = 0
mse_finetuning = 0
rmse_finetuning = 0
size_test = len(test_df["id"].values)
# Load the two spectrograms
for i,id in enumerate(test_df["id"].values):

  spec_label = librosa.load('wav/'+str(id)+'.wav', sr=sample_rate)[0]
  spec_original_model = librosa.load('test_wav_original/'+str(i)+'.wav', sr=sample_rate)[0]
  spec_finetuning_model = librosa.load('test_wav_finetuning/'+str(i)+'.wav', sr=sample_rate)[0]

  split_len = min(len(spec_label),len(spec_original_model),len(spec_finetuning_model))
  spec_label = spec_label[:split_len]
  spec_original_model = spec_original_model[:split_len]
  spec_finetuning_model = spec_finetuning_model[:split_len]

  # Compute the MSE metric
  mse_original += np.mean((spec_label - spec_original_model)**2)
  rmse_original += np.sqrt(mse_original)

  mse_finetuning += np.mean((spec_label - spec_finetuning_model)**2)
  rmse_finetuning += np.sqrt(mse_finetuning)

mse_original = mse_original / size_test
rmse_original = rmse_original / size_test

mse_finetuning = mse_finetuning / size_test
rmse_finetuning = rmse_finetuning / size_test

# Metrics of model BEFORE finetuing in test data
print("### Metrics in test data BEFORE finetuing:")
print("Mean Square Error: (lower is better)",mse_original)
print("Root Mean Square Error (lower is better):",rmse_original)
print("=============================================")
print("### Metrics in test data AFTER finetuing:")
print("Mean Square Error: (lower is better)",mse_finetuning)
print("Root Mean Square Error (lower is better):",rmse_finetuning)

### Metrics in test data BEFORE finetuing:
Mean Square Error: (lower is better) 0.044785238588228825
Root Mean Square Error (lower is better): 2.0110066944004297
### Metrics in test data AFTER finetuing:
Mean Square Error: (lower is better) 0.043612250527366996
Root Mean Square Error (lower is better): 1.97773962268741


# **Test inference**

In [ ]:
!pip install piper-tts

In [ ]:
!echo 'Xin chào! Đây là âm thanh kiểm tra chất lượng của mô hình tổng hợp tiếng nói trên văn bản tiếng Việt được trích xuất sang định dạng ONNX' | \
  piper -m /content/pretrained_vi.onnx --output_file /content/test_original.wav

In [ ]:
!echo 'Xin chào! Đây là âm thanh kiểm tra chất lượng của mô hình tổng hợp tiếng nói trên văn bản tiếng Việt được trích xuất sang định dạng ONNX' | \
  piper -m /content/finetuning_pretrained_vi.onnx --output_file /content/test.wav

In [ ]:
# Âm thanh model gốc
from IPython.display import Audio
Audio("/content/test_original.wav")

In [ ]:
# Âm thanh model finetuning
from IPython.display import Audio
Audio("/content/test.wav")